In [4]:
import json
import requests
import re, time
import numpy as np
import time, datetime
from bs4 import BeautifulSoup
from tqdm import tqdm

In [15]:
def retrieve_html(url):
    """
    Return the raw HTML at the specified URL.

    Args:
        url (string): 

    Returns:
        status_code (integer):
        raw_html (string): the raw HTML content of the response, properly encoded according to the HTTP headers.
    """
    response = requests.get(url)
    return (response.status_code, response.text)
    pass
def get_people_list_data(url, key_words):
    code, text = retrieve_html(url)
    soup = BeautifulSoup(text,'html.parser')
    next_page = set()
    for link in soup.find('div',{'id':'bodyContent'}).find_all('a', href=True):
        tar_url=link.attrs['href']
        if tar_url.startswith('/wiki/') and tar_url.find(':') == -1:
            next_page.add(tar_url)
    data = dict()
    for url in tqdm(next_page):
        time.sleep(0.1)
        code, html = retrieve_html("https://en.wikipedia.org"+url)
        soup = BeautifulSoup(html,'html.parser')
        if html.find('<th scope="row">Born</th>') != -1 and any(key_word in html for key_word in key_words):
            div = soup.find('div', {"class": "mw-parser-output"})
            text = ""
            for p in div.find_all("p", recursive=False):
                tmp = re.sub("(\[[0-9]+\])", "", p.text)
                tmp = re.sub("(\(.*\))", "", tmp).rstrip().strip()
                text += tmp
            data[url[6:]] = text
    return data
def get_all_data(universities):
    data = dict()
    for key, value in universities.items():
        print(key)
        data[key] = get_people_list_data(value[0], value[1])
    with open('data.json', 'w') as fp:
        json.dump(data, fp)

In [16]:
universities = dict()
universities['CMU'] = ("https://en.wikipedia.org/wiki/List_of_Carnegie_Mellon_University_people", \
                       ['Carnegie_Mellon_University', 'CMU'])
universities['UCLA'] = ("https://en.wikipedia.org/wiki/List_of_University_of_California,_Los_Angeles_people", \
                       ['University_of_California,_Los_Angeles', 'University of California at Los Angeles', 'UCLA'])
universities['Stanford'] = ("https://en.wikipedia.org/wiki/List_of_Stanford_University_people", \
                       ['Stanford_University'])
universities['Harvard'] = ("https://en.wikipedia.org/wiki/List_of_Harvard_University_people", \
                           ['Harvard_University'])
universities['Berklee'] = ("https://en.wikipedia.org/wiki/List_of_Berklee_College_of_Music_alumni", \
                           ['Berklee_College_of_Music', 'Berklee'])
get_all_data(universities)

CMU


100%|██████████| 1065/1065 [05:09<00:00,  3.44it/s]


UCLA


100%|██████████| 1629/1629 [08:30<00:00,  3.19it/s]


Stanford


100%|██████████| 2123/2123 [10:48<00:00,  3.27it/s]


Harvard


100%|██████████| 2828/2828 [15:19<00:00,  3.08it/s]


Berklee


100%|██████████| 150/150 [00:41<00:00,  3.65it/s]
